In [64]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, MinMaxScaler

# ignore warnings
import warnings
warnings.filterwarnings("ignore")

from acquire import get_telco_data

In [93]:
telco=get_telco_data()

In [91]:
#Getting just a look at what the data frame looks like by using .head()
telco.head()

,customer_id,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,online_security,online_backup,...,payment_type,years_tenure,has_streaming,is_family,has_phones,has_security_features,Yes,True,Yes,Yes
0,0003-MKNFE,Male,0,No,No,9,Yes,Yes,No,No,...,Mailed check,0.750000,False,False,True,False,0,0,0,0
1,0013-MHZWF,Female,0,No,Yes,9,Yes,No,No,No,...,Credit card (automatic),0.750000,True,False,True,False,0,0,0,0
2,0015-UOCOJ,Female,1,No,No,7,Yes,No,Yes,No,...,Electronic check,0.583333,False,False,True,True,0,1,0,0
3,0023-HGHWL,Male,1,No,No,1,No,No phone service,No,No,...,Electronic check,0.083333,False,False,False,False,1,0,1,1
4,0032-PGELS,Female,0,Yes,Yes,1,No,No phone service,Yes,No,...,Bank transfer (automatic),0.083333,False,True,False,True,1,1,1,1


In [94]:
# Describing the numerical data
telco.describe()

,Unnamed: 0,senior_citizen,tenure,internet_service_type_id,contract_type_id,payment_type_id,monthly_charges,contract_type_id.1,internet_service_type_id.1,payment_type_id.1
count,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000
mean,3521.000000,0.162147,32.371149,1.872923,1.690473,2.315633,64.761692,1.690473,1.872923,2.315633
std,2033.283305,0.368612,24.559481,0.737796,0.833755,1.148907,30.090047,0.833755,0.737796,1.148907
min,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,18.250000,1.000000,1.000000,1.000000
25%,1760.500000,0.000000,9.000000,1.000000,1.000000,1.000000,35.500000,1.000000,1.000000,1.000000
50%,3521.000000,0.000000,29.000000,2.000000,1.000000,2.000000,70.350000,1.000000,2.000000,2.000000
75%,5281.500000,0.000000,55.000000,2.000000,2.000000,3.000000,89.850000,2.000000,2.000000,3.000000
max,7042.000000,1.000000,72.000000,3.000000,3.000000,4.000000,118.750000,3.000000,3.000000,4.000000


In [66]:
# removing excess columns
telco= telco.drop(columns='Unnamed: 0')
telco= telco.drop(columns='internet_service_type_id.1')
telco= telco.drop(columns='payment_type_id')
telco= telco.drop(columns='contract_type_id.1')
telco= telco.drop(columns='payment_type_id.1')
telco= telco.drop(columns= 'contract_type_id')
telco= telco.drop(columns='internet_service_type_id')

In [67]:
#making single variables
telco['years_tenure'] = telco.tenure / 12
telco['has_streaming']= telco["streaming_tv" or "streaming_movies"] == 'Yes'
telco['is_family']=telco["partner" or "dependents"] == 'Yes'
telco['has_phones']= telco['phone_service' or 'multiple_lines']== 'Yes'
telco['has_security_features']= telco['online_security' or 'online_backup'] =='Yes'

In [68]:
#double checking my work
telco.head()

,customer_id,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,online_security,online_backup,...,internet_service_type,payment_type,years_tenure,has_streaming,is_family,has_phones,has_security_features,Yes,True,Yes
0,0003-MKNFE,Male,0,No,No,9,Yes,Yes,No,No,...,DSL,Mailed check,0.750000,False,False,True,False,0,0,0
1,0013-MHZWF,Female,0,No,Yes,9,Yes,No,No,No,...,DSL,Credit card (automatic),0.750000,True,False,True,False,0,0,0
2,0015-UOCOJ,Female,1,No,No,7,Yes,No,Yes,No,...,DSL,Electronic check,0.583333,False,False,True,True,0,1,0
3,0023-HGHWL,Male,1,No,No,1,No,No phone service,No,No,...,DSL,Electronic check,0.083333,False,False,False,False,1,0,1
4,0032-PGELS,Female,0,Yes,Yes,1,No,No phone service,Yes,No,...,DSL,Bank transfer (automatic),0.083333,False,True,False,True,1,1,1


In [69]:
#making dummy variables
telco_dummies = pd.get_dummies(telco.churn, drop_first=True)


telco_dummies.head(3)

,Yes
0,0
1,0
2,0


In [70]:
#concat the dummy variables to my df
telco = pd.concat([telco, telco_dummies], axis=1)
telco.head()

,customer_id,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,online_security,online_backup,...,payment_type,years_tenure,has_streaming,is_family,has_phones,has_security_features,Yes,True,Yes,Yes
0,0003-MKNFE,Male,0,No,No,9,Yes,Yes,No,No,...,Mailed check,0.750000,False,False,True,False,0,0,0,0
1,0013-MHZWF,Female,0,No,Yes,9,Yes,No,No,No,...,Credit card (automatic),0.750000,True,False,True,False,0,0,0,0
2,0015-UOCOJ,Female,1,No,No,7,Yes,No,Yes,No,...,Electronic check,0.583333,False,False,True,True,0,1,0,0
3,0023-HGHWL,Male,1,No,No,1,No,No phone service,No,No,...,Electronic check,0.083333,False,False,False,False,1,0,1,1
4,0032-PGELS,Female,0,Yes,Yes,1,No,No phone service,Yes,No,...,Bank transfer (automatic),0.083333,False,True,False,True,1,1,1,1


In [71]:
telco.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 30 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   customer_id            7043 non-null   object 
 1   gender                 7043 non-null   object 
 2   senior_citizen         7043 non-null   int64  
 3   partner                7043 non-null   object 
 4   dependents             7043 non-null   object 
 5   tenure                 7043 non-null   int64  
 6   phone_service          7043 non-null   object 
 7   multiple_lines         7043 non-null   object 
 8   online_security        7043 non-null   object 
 9   online_backup          7043 non-null   object 
 10  device_protection      7043 non-null   object 
 11  tech_support           7043 non-null   object 
 12  streaming_tv           7043 non-null   object 
 13  streaming_movies       7043 non-null   object 
 14  paperless_billing      7043 non-null   object 
 15  mont

In [43]:
telco.describe()

,senior_citizen,tenure,monthly_charges,years_tenure,Yes
count,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000
mean,0.162147,32.371149,64.761692,2.697596,0.265370
std,0.368612,24.559481,30.090047,2.046623,0.441561
min,0.000000,0.000000,18.250000,0.000000,0.000000
25%,0.000000,9.000000,35.500000,0.750000,0.000000
50%,0.000000,29.000000,70.350000,2.416667,0.000000
75%,0.000000,55.000000,89.850000,4.583333,1.000000
max,1.000000,72.000000,118.750000,6.000000,1.000000


In [78]:
##  This function acquires and prepares the telco data from a local csv, default.
##  Passing cached=False acquires fresh data from sql and writes to csv.
##  Returns the telco df with dummy variables encoding species.


def clean_telco(cached=True):
   
    
    # use my aquire function to read data into a df from a csv file
    df = get_telco_data()
    # drop duplicates
    df.drop_duplicates(inplace=True)
    
    # drop and rename columns
    df= df.drop(columns='Unnamed: 0')
    df= df.drop(columns='internet_service_type_id.1')
    df= df.drop(columns='payment_type_id')
    df= df.drop(columns='contract_type_id.1')
    df= df.drop(columns='payment_type_id.1')
    df= df.drop(columns= 'contract_type_id')
    df= df.drop(columns='internet_service_type_id')
    df['years_tenure'] = df.tenure / 12
    df['has_streaming']= df["streaming_tv" or "streaming_movies"] == 'Yes'
    df['is_family']=df["partner" or "dependents"] == 'Yes'
    df['has_phones']= df['phone_service' or 'multiple_lines']== 'Yes'
    df['has_security_features']= df['online_security' or 'online_backup'] =='Yes'
    df['years_tenure'] = df.tenure / 12
    # create dummy columns for churn
    telco_dummies = pd.get_dummies(df.churn, drop_first=True)
    
    # add dummy columns to df
    df = pd.concat([df, telco_dummies], axis=1)
    # rename dummy columns
    df= df.rename(columns={'Yes': 'is_churn'})
    
    return df

#making my split, train, test data using is_churn
train_validate, test = train_test_split(df, test_size=.2, 
                                        random_state=123,
                                        stratify=df.is_churn
                                       )
train, validate = train_test_split(train_validate, test_size=.3, 
                                 random_state=123,
                                 stratify=train_validate.is_churn
                                        )

In [102]:
#combining my split, train, test data and my clean data into one dataframe
def prep_telco_data():
    df = clean_telco()
    train_validate, test = train_test_split(df, test_size=.2, random_state=123, stratify=df.is_churn)
    train, validate = train_test_split(train_validate, 
                                       test_size=.3, 
                                       random_state=123, 
                                       stratify=train_validate.is_churn)
    return train, validate, test